In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# 2. 필요한 패키지 설치
!pip install faiss-cpu sentence-transformers transformers fastapi uvicorn nest_asyncio pyngrok datasets

In [ ]:
# 3. 주요 라이브러리 로딩
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import faiss
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
# 4. Q&A 모델 로딩
embedding_model = SentenceTransformer('jhgan/ko-sbert-sts')
model_path = "/content/drive/MyDrive/Colab Notebooks/KoGPT/Model/Q&A/Response"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 5. 유사도 임베딩 벡터 및 FAISS 인덱스 생성
qa_texts = [
    "예선대회 결과는 언제 공지되나요?",
    "문제 정답과 점수는 공개되지 않나요?",
    "채점 기준은 어떻게 되나요?",
    "대회 기간을 놓쳤습니다. 어떻게 해야 하나요?",
    "답안 작성이 되지 않습니다.",
    "대회 원서접수를 했는데 문제보기가 되지 않습니다.",
    "대회 기간 중인데 문제보기가 되지 않습니다.",
    "예선대회 진행방식은 어떻게 되나요?",
    "대회 관련 기출 문제들은 블로그 등에 올려도 되나요?",
    "접속자가 많아 접수가 어렵습니다.",
    "신청 기간 이후 추가 접수 가능한가요?",
    "학교에 다니지 않는 사람은 어떤 부문에 참가해야 하나요?",
    "해외거주 중인데 참가 가능한가요?",
    "팀명은 꼭 7자 이내로만 작성해야 하나요?",
    "대회 참가 접수를 확인하고 싶습니다.",
    "대회 원서 접수 시 팀원이 모두 가입해야 하나요?",
    "다른 부문끼리 팀 구성이 가능한가요?",
    "같은 학교 학생끼리만 팀 구성이 가능한가요?",
    "대회 참가 자격이 궁금합니다.",
    "대회 개최 일정은 어떻게 되나요?",
    "대회 관련 문의사항은 어디로 문의하나요?"
]
embeddings = embedding_model.encode(qa_texts, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False).astype('float32')
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

In [ ]:
# 6. 답변 생성 함수
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=100,
        temperature=0.6,
        top_p=0.5,
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.convert_tokens_to_ids("<END>")
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print("generated_text: ", generated_text)
    return generated_text.split("답변: ")[1].strip()

In [ ]:
# 7. FastAPI app 설정
app = FastAPI()

class QARequest(BaseModel):
    question: str

class QAResponse(BaseModel):
    response: str

@app.post("/qa", response_model=QAResponse)
def qa_endpoint(request: QARequest):
    embedding = embedding_model.encode([request.question], normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=False).astype('float32')
    _, indices = index.search(embedding, 1)
    similar_question = qa_texts[indices[0][0]]
    print("similar_question: ", similar_question)

    prompt = f"질문: {similar_question}\n답변: "

    with torch.no_grad():
        response = generate_response(prompt)

    return QAResponse(response=response)

In [ ]:
# 8. Ngrok 실행
ngrok.set_auth_token("***")
ngrok.kill()

public_url = ngrok.connect(3000)
print("🔗 Public URL:", public_url.public_url)

nest_asyncio.apply()
uvicorn.run(app, host="0.0.0.0", port=3000)